# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [37]:
import os
import sys

sys.path.append('..')
from utils import *

# Set your preferred name
USER = "Haoyang"
# Set the data and output directories
DATA_ROOT = '/media/techt/DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Large B-cell Lymphoma'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [38]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [39]:
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)

['TCGA_Lower_Grade_Glioma_(LGG)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Lung_Adenocarcinoma_(LUAD)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 '.DS_Store',
 'CrawlData.ipynb',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Bladder_Cancer_(BLCA)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Cervical_Cancer_(CESC)',
 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Esophageal_

If no match is found, jump directly to GEO in Part 2.2

In [40]:
trait_subdir = "TCGA_Large_Bcell_Lymphoma_(DLBC)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

cohort_dir = os.path.join(dataset_dir, trait_subdir)
clinical_data_file, genetic_data_file = xena_get_relevant_filepaths(cohort_dir)

In [50]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [42]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'age_at_initial_pathologic_diagnosis',
 'b_lymphocyte_genotyping_method',
 'bcr_followup_barcode',
 'bcr_patient_barcode',
 'bcr_sample_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [43]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'age_at_initial_pathologic_diagnosis', 'b_lymphocyte_genotyping_method', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'bone_marrow_biopsy_done', 'bone_marrow_involvement', 'bone_marrow_sample_histology', 'clinical_stage', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_ini

In [44]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [45]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [75, 67, 40, 73, 58],
 'days_to_birth': [-27468, -24590, -14723, -27025, -21330]}

In [46]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [47]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [48]:
gender_col = 'gender'

In [13]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col=age_col, gender_col=gender_col)

In [14]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

13 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('RPL31P11', 2), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
10 input query terms found dup hits:	[('SUGT1P1', 2), ('PTPRVP', 2), ('SNORA62', 3), ('IFITM4P', 7), ('HLA-DRB6', 2), ('FUNDC2P2', 2), ('
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
11 input query terms found dup hits:	[('PIP5K1P1', 2), ('HBD', 2), ('PPP1R2P1', 9), ('HSD17B7P2', 2), ('RPSAP9', 2), ('SNORD68', 2), ('SN
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
15 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('ESPNP', 2), ('HBBP1
158 input query terms found no hit:	[

In [15]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Large B-cell Lymphoma,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-FA-8693-01,1,75,1,-0.016192,-4.254026,-0.531035,-0.309572,-1.605778,0.31609,-1.112794,...,-2.090786,0.486518,-0.748878,-1.587117,4.630667,-1.28139,-2.739077,-0.758427,1.338005,2.226867
TCGA-FA-A4BB-01,1,67,1,-0.649392,-0.784126,-0.531035,-0.116472,-1.052078,-0.07741,-0.430994,...,0.127114,-0.086682,0.042322,1.466783,3.208567,-1.28139,-1.449877,-0.970927,0.541605,2.017767
TCGA-FA-A4XK-01,1,40,1,-1.168292,-2.833526,-0.531035,0.521528,-1.519678,-0.94711,-0.935594,...,-2.090786,1.371818,-0.316278,-1.154517,2.361867,-1.28139,-1.479277,0.767273,0.701205,-1.022333
TCGA-FA-A6HN-01,1,73,1,-0.836392,0.240874,0.045665,-0.090072,-1.722578,-0.85351,-0.945494,...,-1.471986,-0.086682,-0.748878,-0.018817,2.804267,1.02051,-1.772477,-0.042427,0.800105,2.998367
TCGA-FA-A6HO-01,1,58,0,-0.596092,-0.265826,-0.531035,-0.082972,-1.472778,0.23509,0.129906,...,-2.090786,-0.086682,-0.266778,0.725683,3.218567,-1.28139,-1.658377,-1.318127,1.176405,1.092467


In [16]:
print(f"The merged dataset contains {len(merged_data)} samples.")
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

The merged dataset contains 48 samples.
For the feature 'Large B-cell Lymphoma', the least common label is '1' with 48 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Large B-cell Lymphoma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 46.0
  50% (Median): 57.5
  75%: 67.0
Min: 23
Max: 82
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 22 occurrences. This represents 45.83% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [17]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

In [18]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Haoyang/Large-B-cell-Lymphoma/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [19]:
dataset = 'GEO'
trait_subdir = "Large-B-cell-Lymphoma"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE156309', 'GSE173263', 'GSE182362', 'GSE184212', 'GSE197977']

Repeat the below steps for all the accession numbers

In [20]:
cohort = accession_num = "GSE197977"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/media/techt/DATA/GEO/Large-B-cell-Lymphoma/GSE197977/GSE197977_family.soft.gz',
 '/media/techt/DATA/GEO/Large-B-cell-Lymphoma/GSE197977/GSE197977_series_matrix.txt.gz')

### Initial filtering and clinical data preprocessing

In [21]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Tumour immune contexture is a determinant of anti-CD19 CAR T-cell efficacy in large B cell lymphoma"
!Series_summary	"Axicabtagene ciloleucel (axi-cel), an anti-CD19 chimeric antigen receptor (CAR) T-cell therapy approved for treatment of relapsed/refractory large B-cell lymphoma (LBCL), has comparable efficacy across conventional LBCL markers. We analysed whether pre- and posttreatment  tumour immune contexture determines clinical outcomes for axi cel–treated patients in the ZUMA-1 pivotal study. Longitudinal evaluation of the tumour microenvironment (TME) uncovered dynamic patterns that occurred rapidly after axi-cel (within 2 weeks) in responders—pronounced enhancement of T- and myeloid cell signatures and diminution of B cell signature. Clinical response and overall survival associated with high CD8+ T-cell density (Immunoscore) and immune gene expression (Immunosign21) in TME pretreatment, which was paralleled by blood CAR T-cell levels posttreatment. High density o

In [22]:
clinical_data.head()

,!Sample_geo_accession,GSM5935018,GSM5935019,GSM5935020,GSM5935021,GSM5935022,GSM5935023,GSM5935024,GSM5935025,GSM5935026,...,GSM5935082,GSM5935083,GSM5935084,GSM5935085,GSM5935086,GSM5935087,GSM5935088,GSM5935089,GSM5935090,GSM5935091
0,!Sample_characteristics_ch1,visit: SCREENING,visit: SCREENING,visit: SCREENING,visit: SCREENING,visit: SCREENING,visit: SCREENING,visit: SCREENING,visit: SCREENING,visit: SCREENING,...,visit: DAY7-14,visit: BASELINE,visit: DAY7-14,visit: DAY7-14,visit: BASELINE,visit: PROGFCB,visit: DAY7-14,visit: DAY7-14,visit: PROGFCB,visit: BASELINE
1,!Sample_characteristics_ch1,molecular subgroup: GCB,molecular subgroup: NA,molecular subgroup: NA,molecular subgroup: GCB,molecular subgroup: ABC,molecular subgroup: GCB,molecular subgroup: ABC,molecular subgroup: NA,molecular subgroup: NA,...,molecular subgroup: GCB,molecular subgroup: ABC,molecular subgroup: ABC,molecular subgroup: NA,molecular subgroup: GCB,molecular subgroup: GCB,molecular subgroup: NA,molecular subgroup: GCB,molecular subgroup: ABC,molecular subgroup: GCB
2,!Sample_characteristics_ch1,bestresponse: PR,bestresponse: CR,bestresponse: CR,bestresponse: CR,bestresponse: CR,bestresponse: CR,bestresponse: PR,bestresponse: PR,bestresponse: CR,...,bestresponse: CR,bestresponse: CR,bestresponse: CR,bestresponse: CR,bestresponse: CR,bestresponse: PR,bestresponse: CR,bestresponse: CR,bestresponse: CR,bestresponse: CR
3,!Sample_characteristics_ch1,baseline tumour burden (mm2): 8877,baseline tumour burden (mm2): 667,baseline tumour burden (mm2): 2441,baseline tumour burden (mm2): 7700,baseline tumour burden (mm2): 4248,baseline tumour burden (mm2): 3916,baseline tumour burden (mm2): 4205,baseline tumour burden (mm2): 5056,baseline tumour burden (mm2): 355,...,baseline tumour burden (mm2): 9953,baseline tumour burden (mm2): 2584,baseline tumour burden (mm2): 6557,baseline tumour burden (mm2): 1022,baseline tumour burden (mm2): 9371,baseline tumour burden (mm2): 3014,baseline tumour burden (mm2): 1724,baseline tumour burden (mm2): 7991,baseline tumour burden (mm2): 10878,baseline tumour burden (mm2): 431
4,!Sample_characteristics_ch1,worst grade of ne: 3,worst grade of ne: 1,worst grade of ne: 3,worst grade of ne: 1,worst grade of ne: 2,worst grade of ne: 3,worst grade of ne: 0,worst grade of ne: 3,worst grade of ne: 1,...,worst grade of ne: 0,worst grade of ne: 1,worst grade of ne: 1,worst grade of ne: 0,worst grade of ne: 3,worst grade of ne: 0,worst grade of ne: 0,worst grade of ne: 1,worst grade of ne: 3,worst grade of ne: 1


In [23]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['visit: SCREENING',
  'visit: BASELINE',
  'visit: DAY7-14',
  'visit: FCBWK4',
  'visit: PROGFCB'],
 1: ['molecular subgroup: GCB',
  'molecular subgroup: NA',
  'molecular subgroup: ABC',
  'molecular subgroup: UNCLASSIFIED'],
 2: ['bestresponse: PR',
  'bestresponse: CR',
  'bestresponse: SD',
  'bestresponse: PD',
  'bestresponse: ND'],
 3: ['baseline tumour burden (mm2): 8877',
  'baseline tumour burden (mm2): 667',
  'baseline tumour burden (mm2): 2441',
  'baseline tumour burden (mm2): 7700',
  'baseline tumour burden (mm2): 4248',
  'baseline tumour burden (mm2): 3916',
  'baseline tumour burden (mm2): 4205',
  'baseline tumour burden (mm2): 5056',
  'baseline tumour burden (mm2): 355',
  'baseline tumour burden (mm2): 7161',
  'baseline tumour burden (mm2): NA',
  'baseline tumour burden (mm2): 2794',
  'baseline tumour burden (mm2): 792',
  'baseline tumour burden (mm2): 5456',
  'baseline tumour burden (mm2): 39658',
  'baseline tumour burden (mm2): 1997',
  'baseline t

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [24]:

f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. Do remember to separate the value by colon ':' first, as the row header should not be included in the returned value. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Large B-cell Lymphoma\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Large B-cell Lymphoma\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is record

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [25]:
age_row = gender_row = None
convert_age = convert_gender = None

In [26]:
is_gene_availabe = True
trait_row = 1
#age_row = 0
#gender_row = 2

trait_type = 'binary'

In [27]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [28]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    value = value.split(':')[1].strip()
    if value == "Diffuse large B-cell lymphoma (DLBCL)":
        return 1  # Indicating presence of DLBCL
    else:
        return 0  # Indicating absence of DLBCL or unknown


In [29]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM5935018,GSM5935019,GSM5935020,GSM5935021,GSM5935022,GSM5935023,GSM5935024,GSM5935025,GSM5935026,GSM5935027,...,GSM5935082,GSM5935083,GSM5935084,GSM5935085,GSM5935086,GSM5935087,GSM5935088,GSM5935089,GSM5935090,GSM5935091
Large B-cell Lymphoma,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [30]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM5935018,GSM5935019,GSM5935020,GSM5935021,GSM5935022,GSM5935023,GSM5935024,GSM5935025,GSM5935026,GSM5935027,...,GSM5935082,GSM5935083,GSM5935084,GSM5935085,GSM5935086,GSM5935087,GSM5935088,GSM5935089,GSM5935090,GSM5935091
ID,,,,,,,,,,,,,,,,,,,,,
1,1758.44,1370.35,738.43,1368.94,1473.55,1599.26,1634.00,1313.76,2905.59,2388.75,...,989.01,1031.16,25270.46,8902.58,1765.66,2804.63,12446.22,2538.75,1196.21,2293.09
2,249.72,236.35,167.63,654.96,59.57,204.32,120.19,0.00,216.34,226.64,...,0.00,0.00,287.00,110.74,0.00,71.81,0.00,87.84,25.30,124.49
3,288.10,513.03,310.33,313.61,282.08,229.36,534.17,530.56,396.87,406.93,...,224.05,318.89,471.01,470.88,280.99,623.41,567.91,279.40,333.18,471.31
4,626.42,308.31,1564.71,252.45,277.77,1598.12,1124.63,931.18,174.79,488.05,...,168.45,1567.88,605.60,500.25,307.09,580.45,555.98,443.40,298.10,314.97
5,1224.42,187.35,196.91,750.25,114.84,31.30,148.81,0.00,7.88,0.00,...,0.00,50.55,0.00,0.00,494.44,0.00,0.00,17.84,2433.20,449.17


In [31]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [32]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']\n"


If not required, jump directly to the gene normalization step

In [33]:
requires_gene_mapping = True

In [34]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1', '2', '3', '4', '5'], 'ORF': ['A2M', 'ABCB1', 'ABL1', 'ADA', 'AICDA'], 'Accession': ['NM_000014.4', 'NM_000927.3', 'NM_005157.3', 'NM_000022.2', 'NM_020661.1']}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [35]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    {gene_row_ids}
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we are analyzing a gene expression dataset, and find that its row headers are some identifiers related to genes:
    ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
    To get the mapping from those identifiers to actual gene symbols, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the identifiers, and which key stores the gene symbols. Please strictly follow this format in your answer:
    identifier_key = 'key_name1'
    gene_symbol_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '4', '5'], 'ORF': ['A2M', 'ABCB1', 'ABL1', 'ADA', 'AICDA'], 'Accession': ['NM_000014.4', 'NM_000927.3', 'NM_005157.3', 'NM_000022.2', 'NM_020661.1']}
    


In [36]:
if requires_gene_mapping:
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'
    gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

KeyError: "['Gene Symbol'] not in index"

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
genetic_data

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

In [ ]:
if is_available:
    save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')
else:
    save_cohort_info(cohort, JSON_PATH, is_available)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort + '.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

In [ ]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

In [ ]:
condition_ranked_df.head()

In [ ]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

In [ ]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

In [ ]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 4. Discussion and report

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)